# Read outputs of analyze_functionalized for further custom analysis

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import sys
sys.path.append('ext')
import plot_util
import accelerometers

In [ ]:
inputdir = pathlib.Path.cwd() / 'outputdata'
# inputdir is previous outputdir
outputdir = pathlib.Path.cwd() / 'outputdata' / 'explore'
outputdir.mkdir(exist_ok=True)

pathlib.Path.cwd()

In [ ]:
combined = pd.read_csv(inputdir / 'Combined.csv', header=[0, 1], index_col=0)
combined.head()

In [ ]:
combined.reset_index()

In [ ]:
# Print number of trips by category
print('Number of Trips')
combined.groupby('Category', axis=1).count().T[combined.index[0]]

In [ ]:
# Recreate standard combined average plot
combined_bycat_log = combined.map(np.log10).groupby('Category', axis=1).mean()
plot_util.generic_plot(combined_bycat_log.rename(np.log10, axis=0), kind='line', xlabel='Hz', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra by Mode',
                       logx=True, logy=True)

In [ ]:
# Plot all trips for each mode
for cat in combined.columns.levels[0]:
    print(cat)
    plt.figure()
    data = combined[cat].apply(np.log10).rename(np.log10, axis=0)
    plot_util.generic_plot(data, kind='line', xlabel='Hz', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Power Spectra for all Trips, ' + cat,
                       logx=True, logy=True)
    plt.close()

From here down may need customization

In [ ]:
# Look at max for BART above 1 Hz
bslice = combined_bycat_log.loc[combined_bycat_log.index > 1]
bslice['BART'].max()

In [ ]:
bslice.index[bslice['BART'].argmax()]

In [ ]:
# Let's look just at BART and Muni
plot_util.generic_plot(combined_bycat_log.rename(np.log10, axis=0)[['Muni Train', 'BART']], kind='line', xlabel='log10(Hz)', ylabel='log10(W/kg/Hz)',
                       output_directory=outputdir, title='Average Power Spectra by Mode, Selected')

In [ ]:
# What does it look like removing the logy?
plot_util.generic_plot((10**combined_bycat_log).rename(np.log10, axis=0)[['Muni Train', 'BART']], kind='line', xlabel='log10(Hz)', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra by Mode, Unlog-y')

In [ ]:
plot_util.generic_plot((10**combined_bycat_log).rename(np.log10, axis=0)[['Muni Train', 'BART']], kind='line', xlabel='log10(Hz)', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra by Mode, Unlog-y', xlim=[-0.75, 1.5], ylim=[0, 150])

In [ ]:
# What does the raw BART trip set look like?
idx = pd.IndexSlice
subtrips = combined.loc[idx[:], idx['BART']]
plot_util.generic_plot(subtrips.rename(np.log10, axis=0), kind='line', xlabel='log10(Hz)', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Trip Power Spectra by Selected Mode')

In [ ]:
# Log them
plot_util.generic_plot(subtrips.rename(np.log10, axis=0).map(np.log10), kind='line', xlabel='log10(Hz)', ylabel='log10(W/kg/Hz)',
                       output_directory=outputdir, title='Logged Trip Power Spectra by Selected Mode')

In [ ]:
# What does the raw Muni trip set look like?
idx = pd.IndexSlice
subtrips = combined.loc[idx[:], idx['Muni Train']]
plot_util.generic_plot(subtrips.rename(np.log10, axis=0), kind='line', xlabel='log10(Hz)', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Trip Power Spectra by Selected Mode_2')

In [ ]:
allmodes = combined_bycat_log.columns
allmodes

In [ ]:
#motorized_travel_modes = ['BART', 'Muni Train', 'Muni Bus', 'BensCar']
# Map columns using accelerometers.mode_key
mode_cats = set([accelerometers.mode_map(mode) for mode in allmodes])
mode_cats

In [ ]:
motorized_travel_modes = accelerometers.mode_match(allmodes, 'Motorized')

In [ ]:
plot_util.generic_plot(combined_bycat_log[motorized_travel_modes].rename(np.log10, axis=0), kind='line', xlabel='Hz', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra for Motorized Travel Modes',
                       logx=True, logy=True)

In [ ]:
combined_bymode_type = combined_bycat_log.rename(accelerometers.mode_map, axis=1).groupby('Category', axis=1).mean()

In [ ]:
plot_util.generic_plot(combined_bymode_type.rename(np.log10, axis=0), kind='line', xlabel='Hz', ylabel='W/kg/Hz',
                       output_directory=outputdir, title='Average Power Spectra by Mode Type',
                       logx=True, logy=True)

In [ ]:
# Let's integrate over three regimes
bounds = [10**(-0.5), 10**(0.75)]
bounds

In [ ]:
combined_bycat_log.head()

In [ ]:
pow_seg = accelerometers.pow_range(combined_bycat_log)
pow_seg

In [ ]:
plot_util.generic_plot(pow_seg.map(np.log10).T, kind='bar', ylabel='W/kg', unstacked=True, title='Power by Freq. Range & Mode', output_directory=outputdir,
                      logy=True, fontsize=9)

In [ ]:
plot_util.generic_plot(pow_seg[motorized_travel_modes].T, kind='bar', ylabel='W/kg', title='Power by Freq. Range for Motorized Modes', output_directory=outputdir)

In [ ]:
# % of power
pow_seg / pow_seg.sum()

In [ ]:
pow_seg.to_csv(outputdir / 'Power by Freq. Range.csv')

In [ ]:
combined_bycat_log.tail()

In [ ]:
# Look at cum rms
cum_rms = combined_bycat_log.apply(lambda xx: (10**np.interp(fp=xx, xp=xx.index, x=np.arange(0.01, 30, 0.01))).cumsum()**0.5)

In [ ]:
cum_rms_norm = cum_rms.apply(lambda x: x / x.max())
cum_rms_norm = cum_rms_norm.rename(lambda x: np.log10(x * 0.01))
plot_util.generic_plot(cum_rms_norm, 'line', output_directory=outputdir, title='Cum RMS', xlabel='Hz', logx=True)